# 1.Tổng quan 
- Có m giáo viên từ 0 - m-1
- Có n môn từ 0- n-1
- mỗi giáo viên sẽ có danh sách preference của mình A[i] = [....]
- mỗi môn học có load nhất định, cho theo danh sách L = [...], có n phần tử. Load của môn i là L[i]
- có các cặp môn ko thể dạy đồng thời bởi 1 giáo viên  

# 2. Nhập dữ liệu
- số lượng giáo viên m, số lượng môn n
- ma trận preference A chứa các giá trị 0,1, A[i][j] = 1 nếu giáo viên i có thể dạy môn j, i thuộc [0,m-1],j thuộc [0,n-1]
- Danh sách L, L[i] là khối lượng của môn i (i thuộc [0,n-1])
- Số lượng cặp môn đối, k
- B : list of tuples of contradict courses


In [2]:
from ortools.linear_solver import pywraplp


def read_input():
    # N
    m = int(input())
    n = int(input())
    # the preference matrix
    A = []
    for _ in range(0,m):
        row = list(map(int, input().split()))
        A.append(row)
    
    L == list(map(int,input().split()))
    k = int(input())
    B = []
    for _ in range(0,k):
        tup = tuple(map(int, input().split()))
        B.append(tup)
    return m,n,A,L,B



# 3. decision vars
- X[i,j] là bool var xác nhận giáo viên i dạy môn j
- Y[j] là int var trong khoảng [0,n-1],: giáo viên dạy môn j

In [3]:
def create_variables(solver, m,n,A,L,B):
   
    X = {}
    for i in range(0,m):
        for j in range(0,n):
            X[i, j] = solver.BoolVar(f'X[{i},{j}]')
    Y = {}
    for j in range(0,n):
        Y[j]= solver.IntVar(0,m-1,f'Y[{j}]')

    Z= solver.IntVar(0,solver.infinity(),'Z')    
    return X,Y,Z

# 4.constraints

In [7]:
import math
def add_constraints(solver,X,Y,Z ,m,n,A,L,B):
    
    # mỗi môn đc đúng 1 giáo viên dạy
    for j in range(n):
        solver.Add(solver.Sum(X[i, j] for i in range(m)) == 1) 
    
    # môn dạy phải nằm trong preference cùa giáo viên
    for i in range(m):
        for j in range(n):
            solver.Add(X[i,j] <= A[i][j])
    # no conflict
    for t in range(m):
        for a in range(n):
            for b in range(n):
                if (((a,b) in B) or ((b,a) in B)) and a!=b:
                    solver.Add(X[t,a]+X[t,b]<= 1)          
    for i in range(m):
        solver.Add(Z >= solver.Sum(X[i,j]*L[j] for j in range(n)))

In [8]:
def main():
    # Create solver
    solver = pywraplp.Solver.CreateSolver('SCIP')
    
    # Read input
    m = 3
    n = 13
    A= [[1, 0 ,1, 1, 1,0 ,0 ,0 ,1, 0, 1, 0, 0], 
        [1, 1, 1 ,1, 0 ,1 ,1, 1 ,1 ,0, 0 ,0, 0],
        [0 ,1 ,1 ,1 ,0 ,0,0 ,1 ,0 ,1,0 ,1 ,1]]  
    L = [3 ,3 , 4, 3, 4 ,3, 3, 3, 4, 3, 3, 4 ,4] 
    k= 12
    B =[(0, 2), (0, 4), (0, 8), (1, 4), (1, 10), (3, 7), (3, 9), (5, 11), (5, 12), (6, 8), (6, 12)]
    # Create variables
    X,Y,Z = create_variables(solver, m,n,A,L,B)
    
    # Set objective: minimize maximum distance
    solver.Minimize(Z)
    
    # Add constraints
    add_constraints(solver,X,Y,Z, m,n,A,L,B)
    
    # Solve the problem
    status = solver.Solve()
    
    # Output results
    if status == pywraplp.Solver.OPTIMAL:
        print(Z.solution_value())

    else:
        print('No optimal solution found.')

main()

15.0
